In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
import os
api_key=os.getenv("GOOGLE_API_KEY")

In [3]:
from langchain.llms import GooglePalm
llm=GooglePalm(google_api_key=api_key,temperature=0.3)

/home/zigron/anaconda3/envs/db/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
llm("Who are you?")

'I am powered by PaLM 2, which stands for Pathways Language Model 2, a large language model from Google AI.'

In [6]:
from langchain.utilities import SQLDatabase
from langchain_experimental.sql import SQLDatabaseChain

# Set up your database connection
db_user = 'root'
db_password = '123456'
db_host = 'localhost'
db_name = 'atliq_tshirts'
db = SQLDatabase.from_uri(f"mysql+pymysql://{db_user}:{db_password}@{db_host}/{db_name}", sample_rows_in_table_info=3)
# Create an instance of SQLDatabaseChain with llm_wrapper
db_chain = SQLDatabaseChain.from_llm(llm, db, verbose=True)

# Run a query
query = "How many Nike shirts do I have left?"
answer1 = db_chain.run(query)
print(answer1)




> Entering new SQLDatabaseChain chain...
How many Nike shirts do I have left?
SQLQuery:SELECT stock_quantity FROM t_shirts WHERE brand = 'Nike'
SQLResult: [(92,), (84,), (16,), (51,), (34,), (50,), (15,), (57,), (51,), (58,), (65,), (34,), (89,), (18,)]
Answer:34
> Finished chain.
34


In [7]:
query = "SELECT sum(stock_quantity) FROM t_shirts WHERE brand = 'Nike' AND color = 'White' AND size = 'XS'"
qns1 = db_chain.run(query)
qns1



> Entering new SQLDatabaseChain chain...
SELECT sum(stock_quantity) FROM t_shirts WHERE brand = 'Nike' AND color = 'White' AND size = 'XS'
SQLQuery:SELECT sum(stock_quantity) FROM t_shirts WHERE brand = 'Nike' AND color = 'White' AND size = 'XS'
SQLResult: [(Decimal('58'),)]
Answer:58
> Finished chain.


'58'

In [8]:
query = "SELECT SUM(price*stock_quantity) FROM t_shirts WHERE size = 'S'"
qns2 = db_chain.run(query)
qns2



> Entering new SQLDatabaseChain chain...
SELECT SUM(price*stock_quantity) FROM t_shirts WHERE size = 'S'
SQLQuery:SELECT SUM(price*stock_quantity) FROM t_shirts WHERE size = 'S'
SQLResult: [(Decimal('21163'),)]
Answer:21163
> Finished chain.


'21163'

In [9]:
query = "SELECT sum(a.total_amount * ((100-COALESCE(discounts.pct_discount,0))/100)) as total_revenue from(select sum(price*stock_quantity) as total_amount, t_shirt_id from t_shirts where brand = 'Levi'group by t_shirt_id) a left join discounts on a.t_shirt_id = discounts.t_shirt_id"
qns3 = db_chain.run(query)
qns3



> Entering new SQLDatabaseChain chain...
SELECT sum(a.total_amount * ((100-COALESCE(discounts.pct_discount,0))/100)) as total_revenue from(select sum(price*stock_quantity) as total_amount, t_shirt_id from t_shirts where brand = 'Levi'group by t_shirt_id) a left join discounts on a.t_shirt_id = discounts.t_shirt_id
SQLQuery:SELECT sum(a.total_amount * ((100-COALESCE(discounts.pct_discount,0))/100)) as total_revenue from(select sum(price*stock_quantity) as total_amount, t_shirt_id from t_shirts where brand = 'Levi'group by t_shirt_id) a left join discounts on a.t_shirt_id = discounts.t_shirt_id
SQLResult: [(Decimal('16404.000000'),)]
Answer:16404.0
> Finished chain.


'16404.0'

In [10]:
query = "SELECT SUM(price * stock_quantity) FROM t_shirts WHERE brand = 'Levi'"
qns4 = db_chain.run(query)
qns4



> Entering new SQLDatabaseChain chain...
SELECT SUM(price * stock_quantity) FROM t_shirts WHERE brand = 'Levi'
SQLQuery:SELECT SUM(price * stock_quantity) FROM t_shirts WHERE brand = 'Levi'
SQLResult: [(Decimal('16404'),)]
Answer:16404
> Finished chain.


'16404'

In [11]:
query = "SELECT sum(stock_quantity) FROM t_shirts WHERE brand = 'Levi' AND color = 'White'"
qns5 = db_chain.run(query)
qns5



> Entering new SQLDatabaseChain chain...
SELECT sum(stock_quantity) FROM t_shirts WHERE brand = 'Levi' AND color = 'White'
SQLQuery:SELECT sum(stock_quantity) FROM t_shirts WHERE brand = 'Levi' AND color = 'White'
SQLResult: [(Decimal('139'),)]
Answer:139
> Finished chain.


'139'

In [12]:
few_shots = [
    {'Question' : "How many t-shirts do we have left for Nike in XS size and white color?",
     'SQLQuery' : "SELECT sum(stock_quantity) FROM t_shirts WHERE brand = 'Nike' AND color = 'White' AND size = 'XS'",
     'SQLResult': "Result of the SQL query",
     'Answer' : qns1},
    {'Question': "How much is the total price of the inventory for all S-size t-shirts?",
     'SQLQuery':"SELECT SUM(price*stock_quantity) FROM t_shirts WHERE size = 'S'",
     'SQLResult': "Result of the SQL query",
     'Answer': qns2},
    {'Question': "If we have to sell all the Levi’s T-shirts today with discounts applied. How much revenue  our store will generate (post discounts)?" ,
     'SQLQuery' : """SELECT sum(a.total_amount * ((100-COALESCE(discounts.pct_discount,0))/100)) as total_revenue from
(select sum(price*stock_quantity) as total_amount, t_shirt_id from t_shirts where brand = 'Levi'
group by t_shirt_id) a left join discounts on a.t_shirt_id = discounts.t_shirt_id
 """,
     'SQLResult': "Result of the SQL query",
     'Answer': qns3} ,
     {'Question' : "If we have to sell all the Levi’s T-shirts today. How much revenue our store will generate without discount?" ,
      'SQLQuery': "SELECT SUM(price * stock_quantity) FROM t_shirts WHERE brand = 'Levi'",
      'SQLResult': "Result of the SQL query",
      'Answer' : qns4},
    {'Question': "How many white color Levi's shirt I have?",
     'SQLQuery' : "SELECT sum(stock_quantity) FROM t_shirts WHERE brand = 'Levi' AND color = 'White'",
     'SQLResult': "Result of the SQL query",
     'Answer' : qns5
     }
]

In [13]:
from langchain.embeddings import HuggingFaceEmbeddings
embeddings=HuggingFaceEmbeddings(model_name='all-MiniLM-L6-v2')
to_vectorize = [" ".join(example.values()) for example in few_shots]
to_vectorize

/home/zigron/anaconda3/envs/db/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


["How many t-shirts do we have left for Nike in XS size and white color? SELECT sum(stock_quantity) FROM t_shirts WHERE brand = 'Nike' AND color = 'White' AND size = 'XS' Result of the SQL query 58",
 "How much is the total price of the inventory for all S-size t-shirts? SELECT SUM(price*stock_quantity) FROM t_shirts WHERE size = 'S' Result of the SQL query 21163",
 "If we have to sell all the Levi’s T-shirts today with discounts applied. How much revenue  our store will generate (post discounts)? SELECT sum(a.total_amount * ((100-COALESCE(discounts.pct_discount,0))/100)) as total_revenue from\n(select sum(price*stock_quantity) as total_amount, t_shirt_id from t_shirts where brand = 'Levi'\ngroup by t_shirt_id) a left join discounts on a.t_shirt_id = discounts.t_shirt_id\n  Result of the SQL query 16404.0",
 "If we have to sell all the Levi’s T-shirts today. How much revenue our store will generate without discount? SELECT SUM(price * stock_quantity) FROM t_shirts WHERE brand = 'Levi' 

In [14]:
from langchain.vectorstores import FAISS
vectordb=FAISS.from_texts(texts=to_vectorize,embedding=embeddings,metadatas=few_shots)

In [15]:
from langchain.prompts import SemanticSimilarityExampleSelector
example_selector=SemanticSimilarityExampleSelector(
    vectorstore=vectordb,
    k=2,
)

In [16]:
example_selector.select_examples({'Question':'How many adidas tshirts i left with?'})

[{'Question': 'How many t-shirts do we have left for Nike in XS size and white color?',
  'SQLQuery': "SELECT sum(stock_quantity) FROM t_shirts WHERE brand = 'Nike' AND color = 'White' AND size = 'XS'",
  'SQLResult': 'Result of the SQL query',
  'Answer': '58'},
 {'Question': "How many white color Levi's shirt I have?",
  'SQLQuery': "SELECT sum(stock_quantity) FROM t_shirts WHERE brand = 'Levi' AND color = 'White'",
  'SQLResult': 'Result of the SQL query',
  'Answer': '139'}]

In [17]:
from langchain.chains.sql_database.prompt import PROMPT_SUFFIX, _mysql_prompt

In [18]:
print(_mysql_prompt)

You are a MySQL expert. Given an input question, first create a syntactically correct MySQL query to run, then look at the results of the query and return the answer to the input question.
Unless the user specifies in the question a specific number of examples to obtain, query for at most {top_k} results using the LIMIT clause as per MySQL. You can order the results to return the most informative data in the database.
Never query for all columns from a table. You must query only the columns that are needed to answer the question. Wrap each column name in backticks (`) to denote them as delimited identifiers.
Pay attention to use only the column names you can see in the tables below. Be careful to not query for columns that do not exist. Also, pay attention to which column is in which table.
Pay attention to use CURDATE() function to get the current date, if the question involves "today".

Use the following format:

Question: Question here
SQLQuery: SQL Query to run
SQLResult: Result of

In [19]:
print(PROMPT_SUFFIX)

Only use the following tables:
{table_info}

Question: {input}


In [20]:
from langchain.prompts.prompt import PromptTemplate
example_prompt=PromptTemplate(
    input_variables=["Question", "SQLQuery", "SQLResult","Answer",],
    template="\nQuestion: {Question}\nSQLQuery: {SQLQuery}\nSQLResult: {SQLResult}\nAnswer: {Answer}",
)

In [21]:
from langchain.prompts import FewShotPromptTemplate
few_shot_prompt=FewShotPromptTemplate(
    example_selector=example_selector,
    example_prompt=example_prompt,
    prefix=_mysql_prompt,
    suffix=PROMPT_SUFFIX,
    input_variables=["input", "table_info", "top_k"],
)

In [22]:
few_shot_prompt

FewShotPromptTemplate(input_variables=['input', 'table_info', 'top_k'], output_parser=None, partial_variables={}, examples=None, example_selector=SemanticSimilarityExampleSelector(vectorstore=<langchain.vectorstores.faiss.FAISS object at 0x731e7f9bb790>, k=2, example_keys=None, input_keys=None), validate_template=True, example_prompt=PromptTemplate(input_variables=['Question', 'SQLQuery', 'SQLResult', 'Answer'], output_parser=None, partial_variables={}, template='\nQuestion: {Question}\nSQLQuery: {SQLQuery}\nSQLResult: {SQLResult}\nAnswer: {Answer}', template_format='f-string', validate_template=True), suffix='Only use the following tables:\n{table_info}\n\nQuestion: {input}', example_separator='\n\n', prefix='You are a MySQL expert. Given an input question, first create a syntactically correct MySQL query to run, then look at the results of the query and return the answer to the input question.\nUnless the user specifies in the question a specific number of examples to obtain, query f

In [23]:
db_chain = SQLDatabaseChain.from_llm(llm, db, verbose=True,prompt=few_shot_prompt)

In [24]:
db_chain("How many white color levis tshirts i have?")



> Entering new SQLDatabaseChain chain...
How many white color levis tshirts i have?
SQLQuery:SELECT sum(stock_quantity) FROM t_shirts WHERE brand = 'Levi' AND color = 'White'
SQLResult: [(Decimal('139'),)]
Answer:139
> Finished chain.


{'query': 'How many white color levis tshirts i have?', 'result': '139'}

In [25]:
db_chain("How much is the price of the inventory for all small size t-shirts?")



> Entering new SQLDatabaseChain chain...
How much is the price of the inventory for all small size t-shirts?
SQLQuery:SELECT sum(price * stock_quantity) FROM t_shirts WHERE size = 'S'
SQLResult: [(Decimal('21163'),)]
Answer:21163
> Finished chain.


{'query': 'How much is the price of the inventory for all small size t-shirts?',
 'result': '21163'}